# Linear Regression

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import mglearn

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 선형 회귀

[선형 회귀](https://kimdingko-world.tistory.com/101)

모델 | 설명
:--- |:---
일반 선형 회귀 | 예측값과 측정값의 오차를 최소화할 수 있도록 회귀 계수를 최적화
릿지(Ridge) | 선형 회귀에 L2 규제를 추가한 모델. L2 규제는 상대적으로 큰 회귀 계수값을 더 작게 만드는 규제 모델
라쏘(Lasso) | 선형 회귀에 L1 규제를 추가한 모델. L1 규체즌 영향력이 작은 피처의 회귀 계수값을 0으로 만드는 규제 모델(피처 선택 기능)
엘라스틱넷(ElasticNet) | L2 규제와 L1 규제를 결합한 모델. 주로 피처가 많은 데이터 셋에 적용
로지스틱 회귀(Logistic Regression) | 분류에 사용되는 선형 회귀 모델

### 1.1 경사 하강법

#### 1.1.1 데이터 생성

In [ ]:
np.random.seed(123)

# y = 4X + 6 + noise(random)
X = 2 * np.random.rand(100,1)
y = 6 + 4 * X + np.random.randn(100,1)

plt.scatter(X, y)
plt.show()

#### 1.1.2 가중치 업데이트 함수 정의

In [ ]:
# 가중치 업데이트 함수 정의
def get_weight_updates(w1, w0, X, y, learning_rate=0.01):
    N = len(y)
    
    # w1_update, w0_update 0으로 초기화
    w1_update = np.zeros_like(w1)
    w0_update = np.zeros_like(w0)
    
    # 예측값 계산 & 오차 계산(실제값 - 예측값)
    y_pred = np.dot(X, w1.T) + w0
    diff = y-y_pred
         
    # w0_update를 dot 행렬 연산으로 구하기 위해 모두 1값을 가진 행렬 생성 
    w0_factors = np.ones((N,1))

    # w1과 w0을 업데이트할 w1_update와 w0_update 계산
    w1_update = -(2/N)*learning_rate*(np.dot(X.T, diff))
    w0_update = -(2/N)*learning_rate*(np.dot(w0_factors.T, diff))    
    
    return w1_update, w0_update

#### 1.1.3 경사하강법 적용 함수 정의

In [ ]:
# w1, w0 업데이트: iters 만큼 반복 적용
def gradient_descent_steps(X, y, iters=10000):
    # w0, w1초기화
    w0 = np.zeros((1,1))
    w1 = np.zeros((1,1))
    
    # w1, w0 업데이트: iters 만큼 반복 적용
    for ind in range(iters):
        w1_update, w0_update = get_weight_updates(w1, w0, X, y, learning_rate=0.01)
        w1 = w1 - w1_update
        w0 = w0 - w0_update
              
    return w1, w0

#### 1.1.4 비용 구하는 함수 정의

In [ ]:
def get_cost(y, y_pred):
    N = len(y) 
    cost = np.sum(np.square(y - y_pred))/N
    return cost

#### 1.1.5 경사하강법 적용

In [ ]:
# y = 4X + 6

w1, w0 = gradient_descent_steps(X, y, iters=1000)
print('w1:{:.3f} w0:{:.3f}'.format(w1[0,0], w0[0,0]))

y_pred = w1[0,0] * X + w0
print('Gradient Descent Total Cost:{:.4f}'.format(get_cost(y, y_pred)))

In [ ]:
plt.scatter(X, y)
plt.plot(X,y_pred)
plt.title('경사하강법 적용')
plt.show()

#### 1.1.6 확률적 경사하강법(SGD) 적용 함수 정의

In [ ]:
def stochastic_gradient_descent_steps(X, y, batch_size=10, iters=1000):
    # w0, w1초기화
    w0 = np.zeros((1,1))
    w1 = np.zeros((1,1))
    
    for ind in range(iters):
        np.random.seed(ind)
        # sample_X, sample_y: batch_size만큼 랜덤 데이터 추출
        stochastic_random_index = np.random.permutation(X.shape[0])
        sample_X = X[stochastic_random_index[0:batch_size]]
        sample_y = y[stochastic_random_index[0:batch_size]]
        
        # w1, w0 업데이트: 추출된 부분 데이터 사용(sample_X, sample_y)
        w1_update, w0_update = get_weight_updates(w1, w0, sample_X, sample_y, learning_rate=0.01)
        w1 = w1 - w1_update
        w0 = w0 - w0_update
    
    return w1, w0

#### 1.1.7 확률적 경사하강법(SGD) 적용

In [ ]:
# y = 4X + 6

w1, w0 = stochastic_gradient_descent_steps(X, y, iters=1000)
print('w1:{:.3f} w0:{:.3f}'.format(w1[0,0], w0[0,0]))

y_pred = w1[0,0] * X + w0
print('Stochastic Gradient Descent Total Cost:{:.4f}'.format(get_cost(y, y_pred)))

In [ ]:
plt.scatter(X, y)
plt.plot(X,y_pred)
plt.title('확률적 경사하강법(SGD) 적용')
plt.show()

### 1.2 선형 회귀 적용

In [ ]:
# 데이터 로딩
df = pd.read_csv('data/boston.csv')
X = df.drop('target', axis=1).values
y = df['target'].values

df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score

In [ ]:
# 데이터 분할
X_train , X_test , y_train , y_test = train_test_split(X , y, random_state=123)

# Linear Regression: 학습
lr = LinearRegression()
lr.fit(X_train ,y_train )

# Linear Regression: 예측, 평가
pred = lr.predict(X_test)
mse  = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)

In [ ]:
print('MSE:',  mse)
print('RMSE:', rmse)
print('R squared score:', r2_score(y_test, pred))

In [ ]:
# 절편
lr.intercept_

In [ ]:
# 회귀 계수
lr.coef_

In [ ]:
# 회귀 계수 정렬
coeff = pd.Series(data=np.round(lr.coef_, 1), index=df.drop('target',axis=1).columns )
coeff.sort_values(ascending=False)

In [ ]:
# cross_val_score 적용
from sklearn.model_selection import cross_val_score

lr = LinearRegression()

neg_mse_scores = cross_val_score(lr, X, y, scoring='neg_mean_squared_error', cv=5)

rmse_scores    = np.sqrt(-1 * neg_mse_scores)

avg_rmse = np.mean(rmse_scores)

In [ ]:
print('5 folds 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print('5 folds 의 개별 RMSE scores : ', np.round(rmse_scores, 2))
print('5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))

# 2 Polynomial regression

- 언더 피팅
- 오버 피팅

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

### 2.1 데이터 생성

In [ ]:
def true_fun(X):
    return np.cos(1.5 * np.pi * X)

In [ ]:
np.random.seed(123)
n_samples = 30
X = np.sort(np.random.rand(n_samples))

# Cosine + Noise
y = true_fun(X) + np.random.randn(n_samples) * 0.1

### 2.2 다항 회귀 적용

In [ ]:
plt.figure(figsize=(14, 5))
degrees = [1, 4, 15]

# 다항 회귀의 차수(degree)를 1, 4, 15로 각각 변화시키면서 비교합니다. 
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())
    
    # 개별 degree별로 Polynomial 변환합니다. 
    polynomial_features = PolynomialFeatures(degree=degrees[i], include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X.reshape(-1, 1), y)
    
    # 교차 검증으로 다항 회귀를 평가합니다. 
    scores = cross_val_score(pipeline, X.reshape(-1,1), y,scoring="neg_mean_squared_error", cv=10)
    coefficients = pipeline.named_steps['linear_regression'].coef_
    print('\nDegree {0} 회귀 계수는 {1} 입니다.'.format(degrees[i], np.round(coefficients),2))
    print('Degree {0} MSE 는 {1:.2f} 입니다.'.format(degrees[i] , -1*np.mean(scores)))
    
    # 0 부터 1까지 테스트 데이터 세트를 100개로 나눠 예측을 수행합니다. 
    # 테스트 데이터 세트에 회귀 예측을 수행하고 예측 곡선과 실제 곡선을 그려서 비교합니다.  
    X_test = np.linspace(0, 1, 100)
    # 예측값 곡선
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model") 
    # 실제 값 곡선
    plt.plot(X_test, true_fun(X_test), '--', label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    
    plt.xlabel("x"); plt.ylabel("y"); plt.xlim((0, 1)); plt.ylim((-2, 2)); plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(degrees[i], -scores.mean(), scores.std()))

plt.show()

# 3 Regularized Linear Models – Ridge, Lasso, ElasticNet

### 3.1 Ridge regression

In [ ]:
# 데이터 로딩
df = pd.read_csv('data/boston.csv')
X = df.drop('target', axis=1).values
y = df['target'].values
df

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

In [ ]:
ridge = Ridge(alpha = 10)
neg_mse_scores = cross_val_score(ridge, X, y, scoring='neg_mean_squared_error', cv=5)
rmse_scores  = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

In [ ]:
print('5 folds 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 3))
print('5 folds 의 개별 RMSE scores : ', np.round(rmse_scores,3))
print('5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))

In [ ]:
# Ridge에 사용될 alpha 파라미터의 값들을 정의
alphas = [0 , 0.1 , 1 , 10 , 100]

# alphas list 값을 iteration하면서 alpha에 따른 평균 rmse 구함.
for alpha in alphas :
    ridge = Ridge(alpha=alpha)
    
    #cross_val_score를 이용하여 5 fold의 평균 RMSE 계산
    neg_mse_scores = cross_val_score(ridge, X, y, scoring='neg_mean_squared_error', cv=5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    print('alpha {} 일 때 5 folds 의 평균 RMSE: {:.3f}'.format(alpha, avg_rmse))

In [ ]:
fig , axs = plt.subplots(figsize=(18,6) , nrows=1 , ncols=5)
coeff_df = pd.DataFrame()

# alphas 리스트 값을 차례로 입력해 회귀 계수 값 시각화 및 데이터 저장. pos는 axis의 위치 지정
for pos , alpha in enumerate(alphas) :
    ridge = Ridge(alpha = alpha)
    ridge.fit(X, y)
    
    # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가.  
    coeff = pd.Series(data=ridge.coef_ , index=df.drop('target',axis=1).columns )
    colname='alpha:'+ str(alpha)
    coeff_df[colname] = coeff
    
    # 막대 그래프로 각 alpha 값에서의 회귀 계수를 시각화. 회귀 계수값이 높은 순으로 표현
    coeff = coeff.sort_values(ascending=False)
    axs[pos].set_title(colname)
    axs[pos].set_xlim(-3,6)
    sns.barplot(x=coeff.values , y=coeff.index, ax=axs[pos])

plt.show()

In [ ]:
ridge_alphas = [0 , 0.1 , 1 , 10 , 100]
sort_column = 'alpha:'+ str(ridge_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

### 3.2 Lasso regression

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

#### Ridge, Lasso, ElasticNet 모델 적용 함수 정의

In [ ]:
# alpha값에 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DataFrame으로 반환

def get_linear_reg_eval(model_name, params=None, X_n=None, y_n=None ):
    coeff_df = pd.DataFrame()
    
    print('####### ', model_name , '#######')
    
    for param in params:
        if   model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
        
        neg_mse_scores = cross_val_score(model, X_n, y_n, scoring='neg_mean_squared_error', cv=5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {}일 때 5 폴드 세트의 평균 RMSE: {:.3f}'.format(param, avg_rmse))
        
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        model.fit(X_n , y_n)
        # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가. 
        coeff = pd.Series(data=model.coef_ , index=df.drop('target',axis=1).columns )
        colname='alpha:'+ str(param)
        coeff_df[colname] = coeff
        
    return coeff_df

In [ ]:
# 라쏘에 사용될 alpha 파라미터의 값들을 정의하고 get_linear_reg_eval() 함수 호출
lasso_alphas = [0.07, 0.1, 0.5, 1, 3]

coeff_lasso_df = get_linear_reg_eval('Lasso', params=lasso_alphas, X_n=X, y_n=y)

In [ ]:
# 반환된 coeff_lasso_df를 첫번째 컬럼순으로 내림차순 정렬하여 회귀계수 DataFrame출력
sort_column = 'alpha:'+ str(lasso_alphas[0])
coeff_lasso_df.sort_values(by=sort_column, ascending=False)

### 3.3 ElasticNet regression

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
# 엘라스틱넷에 사용될 alpha 파라미터의 값들을 정의하고 get_linear_reg_eval() 함수 호출
# l1_ratio는 0.7로 고정
elastic_alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df = get_linear_reg_eval('ElasticNet', params=elastic_alphas, X_n=X, y_n=y)

In [ ]:
# 반환된 coeff_elastic_df를 첫번째 컬럼순으로 내림차순 정렬하여 회귀계수 DataFrame출력
sort_column = 'alpha:'+ str(elastic_alphas[0])
coeff_elastic_df.sort_values(by=sort_column, ascending=False)

# 4 로지스틱 회귀

- Sigmoid function
- Softmax function

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 로딩
cancer = load_breast_cancer()

In [ ]:
# Scaling data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(cancer.data)

In [ ]:
# 데이터 분할
X_train , X_test, y_train , y_test = train_test_split(data_scaled, cancer.target, random_state=123)

In [ ]:
# 로지스틱 회귀: 학습, 예측
from sklearn.metrics import accuracy_score, roc_auc_score

# 로지스틱 회귀를 이용하여 학습 및 예측 수행. 
lr_clf = LogisticRegression(C=1)
#lr_clf = LogisticRegression(C=0.01)
#lr_clf = LogisticRegression(C=10)

lr_clf.fit(X_train, y_train)

lr_preds = lr_clf.predict(X_test)

# accuracy와 roc_auc 측정
print('accuracy: {:0.3f}'.format(accuracy_score(y_test, lr_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_test , lr_preds)))

---

In [ ]:
# End of file